In [1]:
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)

from time import time_ns

from amftrack.util.dbx import upload_folders, load_dbx, download
from datetime import datetime
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    directory_scratch,
    directory_project,
    run_parallel_stitch,
)
from amftrack.util.dbx import (
    read_saved_dropbox_state,
    get_dropbox_folders_prince,
    get_dropbox_folders_prince,
    save_dropbox_state,
    get_dropbox_folders_general_recursive,
)

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
plates 

In [2]:
# get_surface_area
plates = ['1048_20220506']

In [3]:
# vasilis
plates = ["777_20230328", "772_20230317", "771_20230411"]

In [2]:
# more A5
plates = [
"360_20230926",
"361_20230926",
"363_20230926",
"326_20230926",
"374_20230926",
"372_20230929",
"377_20230929",
"350_20231003",
"358_20231003",
"365_20231003",
"370_20231003",
"371_20231003",
"379_20231003",
"359_20231005",
"362_20231005",
"389_20231005",
"369_20231005",
"380_20231006",
"364_20231009",
"368_20231009"
]

In [5]:
#A5
plates = [
"310_20230830"
]

In [7]:
# for reROI
plates = [
    "28_20230227",
    "206_20230303",
    "202_20230314",
    "218_20230227",
    "219_20230307",
    "229_20230330",
]
plates += [
    "249_20230324",
    "241_20230403",
    "252_20230316",
    "248_20230320",
]

In [13]:
# for P
plates = [
    '478_20230814', '468_20230809', '447_20230821', '487_20230922',
       '492_20230901', '471_20230821',
    '486_20231009',
       '494_20230908',
          '482_20230908',
          '495_20231013',
          '463_20231013',
       '491_20231013', 
          '481_20231005', '483_20231005', 
          '490_20231003',
       '485_20230929',
          "310_20230830"
         ]

In [2]:
plates = [
          "310_20230830"
         ]

In [2]:
# plates = [
# "241_20230403",
# "247_20230422"
# ]
# runnning on 31/05
plates = [
    "557_20230120",
    "553_20230106",
    "551_20220102",
    "545_20220115",
    "540_20221227",
    "539_20221228",
    "533_20230109",
    "532_20230104",
    "531_20221222",
    "530_20230115",
]

In [5]:
# for chicory
plates = [
    "896_20230424",
    "894_20230516",
    "891_20230519",
    "886_20230516",
    "870_20230427",
    "868_20230504",
    "796_20230419",
    "795_20230403",
    "764_20221219",
]

In [2]:
# for P
plates = [
    # "416_20230705",
    # "427_20230707",
    # "420_20230705",
    # "431_20230627",
    # "474_20230807",
    # "460_20230807",
    # "464_20230807",
    "440_20230723",
    # "436_20230717",
    # "443_20230720",
    # "439_20230804",
    # "470_20230709",
]

In [3]:
directory_targ = directory_project
directory_project

'/projects/0/einf914/data/'

In [3]:
directory_targ = os.path.join(directory_scratch, "stitch_temp") + "/"

In [4]:
dir_drop = "DATA/PRINCE"
all_folders_drop = get_dropbox_folders_prince("/DATA/PRINCE")
folders_drop = all_folders_drop.loc[all_folders_drop["unique_id"].isin(plates)]
update_plate_info(directory_targ, local=True, strong_constraint=False)
all_folders = get_current_folders(directory_targ, local=True)
# if len(all_folders) > 0:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders_drop2 = folders_drop.loc[~folders_drop["folder"].isin(folders["folder"])]
folders_drop3 = folders_drop2.loc[~folders_drop2["folder"].str.contains("Analysis")]

2023-11-17 14:17:48,930-[INFO]- dropbox:390 -> Refreshing access token.
2023-11-17 14:17:49,175-[INFO]- dropbox:474 -> Request to files/list_folder
2023-11-17 14:17:50,671-[INFO]- dropbox:474 -> Request to files/list_folder/continue


sizes:   0%|          | 0/728 [00:00<?, ?it/s]

2023-11-17 14:17:51,424-[INFO]- dropbox:390 -> Refreshing access token.
2023-11-17 14:17:51,577-[INFO]- dropbox:474 -> Request to files/list_folder
2023-11-17 14:17:52,069-[INFO]- dropbox:390 -> Refreshing access token.
2023-11-17 14:17:52,214-[INFO]- dropbox:474 -> Request to files/list_folder
2023-11-17 14:17:52,757-[INFO]- dropbox:390 -> Refreshing access token.
2023-11-17 14:17:52,913-[INFO]- dropbox:474 -> Request to files/list_folder
2023-11-17 14:17:53,358-[INFO]- dropbox:390 -> Refreshing access token.
2023-11-17 14:17:53,510-[INFO]- dropbox:474 -> Request to files/list_folder
2023-11-17 14:17:53,845-[INFO]- dropbox:390 -> Refreshing access token.
2023-11-17 14:17:54,008-[INFO]- dropbox:474 -> Request to files/list_folder
2023-11-17 14:17:54,358-[INFO]- dropbox:390 -> Refreshing access token.
2023-11-17 14:17:54,519-[INFO]- dropbox:474 -> Request to files/list_folder
2023-11-17 14:17:54,842-[INFO]- dropbox:390 -> Refreshing access token.
2023-11-17 14:17:54,993-[INFO]- dropbox:

analysed:   0%|          | 0/5183 [00:00<?, ?it/s]

In [5]:
folders_drop3

,folder,size,change_date,tot_path_drop,Plate,CrossDate,unique_id
4983,20231007_1228_Plate16,NaN,0,data/prince/365_20231003/20231007_1228_plate16,365,20231003,365_20231003


In [4]:
update_plate_info(directory_targ, local=True, strong_constraint=False)
all_folders = get_current_folders(directory_targ, local=True)
# if len(all_folders) > 0:
# folders = all_folders.loc[all_folders["unique_id"].isin(plates)]

analysed:   0%|          | 0/243 [00:00<?, ?it/s]

In [10]:
plates = all_folders[all_folders['medium'].isin(["'001P100N200C'", "'001P100N100C'"])]['unique_id'].unique()

In [5]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
print(len(folders))
# folders = folders[folders["/Analysis/nx_graph_pruned_labeled2.p"] == False]
# print(len(folders))

184


In [6]:

name_job = "test_cache"
folders = folders.loc[folders["/Img/TileConfiguration.txt.registered"] == True]
num_parallel = 1
time = "20:00"
hyph_width = 30
perc_low = 85
perc_high = 99.5
minlow = 10
minhigh = 70

args = [hyph_width, perc_low, perc_high, minlow, minhigh, directory_targ]
run_parallel(
    "extract_skel_cache.py",
    args,
    folders,
    num_parallel,
    time,
    "skeletonization",
    cpus=128,
    node="rome",
    name_job=name_job,
)

Submitted batch job 4468396
Submitted batch job 4468397
Submitted batch job 4468398
Submitted batch job 4468399
Submitted batch job 4468400
Submitted batch job 4468401
Submitted batch job 4468402
Submitted batch job 4468403
Submitted batch job 4468404
Submitted batch job 4468405
Submitted batch job 4468406
Submitted batch job 4468407
Submitted batch job 4468408
Submitted batch job 4468409
Submitted batch job 4468410
Submitted batch job 4468411
Submitted batch job 4468412
Submitted batch job 4468413
Submitted batch job 4468414
Submitted batch job 4468415
Submitted batch job 4468416
Submitted batch job 4468417
Submitted batch job 4468418
Submitted batch job 4468419
Submitted batch job 4468420
Submitted batch job 4468421
Submitted batch job 4468422
Submitted batch job 4468423
Submitted batch job 4468424
Submitted batch job 4468425
Submitted batch job 4468426
Submitted batch job 4468427
Submitted batch job 4468428
Submitted batch job 4468429
Submitted batch job 4468430
Submitted batch job 

In [7]:
folders = folders.loc[folders["/Img/TileConfiguration.txt.registered"] == True]
num_parallel = 100
time = "6:00:00"
hyph_width = 30
perc_low = 85
perc_high = 99.5
minlow = 10
minhigh = 70

args = [hyph_width, perc_low, perc_high, minlow, minhigh, directory_targ]
run_parallel(
    "extract_skel_cache_follow.py",
    args,
    folders,
    num_parallel,
    time,
    "skeletonization",
    cpus=128,
    node="fat_rome",
    name_job=name_job,
)

Submitted batch job 4469242
Submitted batch job 4469243


In [ ]:
name_job= "fix_stitch"
num_parallel = 50
time = "40:00"
if len(folders) > 0:
    run_parallel_stitch(
        directory_targ,
        folders,
        num_parallel,
        time,
        cpus=128,
        node="fat",
        name_job=name_job,
    )

/projects/0/einf914/data//20230822_1247_Plate15/Img/Img_r01_c04.tif
/projects/0/einf914/data//20230822_1247_Plate15/Img/Img_r01_c06.tif
/projects/0/einf914/data//20230822_1247_Plate15/Img/Img_r01_c08.tif
/projects/0/einf914/data//20230822_1247_Plate15/Img/Img_r01_c10.tif
/projects/0/einf914/data//20230822_1247_Plate15/Img/Img_r01_c11.tif
/projects/0/einf914/data//20230822_1247_Plate15/Img/Img_r02_c02.tif
/projects/0/einf914/data//20230822_1247_Plate15/Img/Img_r02_c04.tif
/projects/0/einf914/data//20230822_1247_Plate15/Img/Img_r02_c10.tif
/projects/0/einf914/data//20230822_1247_Plate15/Img/Img_r02_c11.tif
/projects/0/einf914/data//20230822_1247_Plate15/Img/Img_r02_c12.tif
/projects/0/einf914/data//20230822_1247_Plate15/Img/Img_r02_c13.tif
/projects/0/einf914/data//20230822_1247_Plate15/Img/Img_r02_c14.tif
/projects/0/einf914/data//20230822_1247_Plate15/Img/Img_r02_c15.tif
/projects/0/einf914/data//20230822_1247_Plate15/Img/Img_r03_c03.tif
/projects/0/einf914/data//20230822_1247_Plate15/

In [6]:
all_folders.columns

Index(['total_path', 'Plate', 'PrincePos', 'root', 'strain', 'medium',
       'fungalC', 'split', 'Pbait', 'CrossDate', 'Temperature',
       '/Img/TileConfiguration.txt.registered',
       '/Analysis/skeleton_compressed.mat',
       '/Analysis/skeleton_masked_compressed.mat',
       '/Analysis/skeleton_pruned_compressed.mat', '/Analysis/transform.mat',
       '/Analysis/transform_corrupt.mat',
       '/Analysis/skeleton_realigned_compressed.mat',
       '/Analysis/nx_graph_pruned.p', '/Analysis/nx_graph_pruned_width.p',
       '/Analysis/nx_graph_pruned_labeled.p', 'date', 'folder', 'unique_id',
       'datetime'],
      dtype='object')

In [9]:
plates = [
    "522_20230115",
    "514_20221206",
    "513_20221205",
    "506_20221124",
    "503_20221124",
    "472_20221003",
]

In [10]:
import shutil

for path in folders["total_path"]:
    shutil.rmtree(path)

In [ ]:
dir_drop = "/DATA/PRINCE"
all_folders_drop = get_dropbox_folders_prince(dir_drop)
folders_drop = all_folders_drop.loc[all_folders_drop["unique_id"].isin(plates)]
# folders_drop

2023-09-03 11:32:24,355-[INFO]- dropbox:390 -> Refreshing access token.
2023-09-03 11:32:24,528-[INFO]- dropbox:474 -> Request to files/list_folder
2023-09-03 11:32:25,714-[INFO]- dropbox:474 -> Request to files/list_folder/continue


sizes:   0%|          | 0/899 [00:00<?, ?it/s]

2023-09-03 11:32:26,776-[INFO]- dropbox:390 -> Refreshing access token.
2023-09-03 11:32:26,924-[INFO]- dropbox:474 -> Request to files/list_folder
2023-09-03 11:32:27,479-[INFO]- dropbox:390 -> Refreshing access token.
2023-09-03 11:32:27,629-[INFO]- dropbox:474 -> Request to files/list_folder
2023-09-03 11:32:28,078-[INFO]- dropbox:390 -> Refreshing access token.
2023-09-03 11:32:28,226-[INFO]- dropbox:474 -> Request to files/list_folder
2023-09-03 11:32:28,647-[INFO]- dropbox:390 -> Refreshing access token.
2023-09-03 11:32:28,784-[INFO]- dropbox:474 -> Request to files/list_folder
2023-09-03 11:32:29,141-[INFO]- dropbox:390 -> Refreshing access token.
2023-09-03 11:32:29,280-[INFO]- dropbox:474 -> Request to files/list_folder
2023-09-03 11:32:29,900-[INFO]- dropbox:390 -> Refreshing access token.
2023-09-03 11:32:30,061-[INFO]- dropbox:474 -> Request to files/list_folder
2023-09-03 11:32:30,643-[INFO]- dropbox:390 -> Refreshing access token.
2023-09-03 11:32:30,790-[INFO]- dropbox:

In [ ]:
folders_drop

In [6]:
folders_drop["Plate"].unique()

array(['908'], dtype=object)

In [6]:
folders

NameError: name 'folders' is not defined

In [17]:
name_job = "transfer_analysis"
run_parallel_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop,
    50,
    "4:00:00",
    "staging",
    cpus=1,
    node="staging",
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3472687


In [9]:
from amftrack.util.sys import (
    path_code,
    temp_path,
    slurm_path,
    slurm_path_transfer,
    conda_path,
)

os.path.exists(os.path.join(conda_path, "envs", "amftrack"))

True

In [5]:
plates = ["558_20230109"]

In [16]:
os.path.join(conda_path, "envs", "amftrack")

'/home/cbisot/miniconda3/envs/amftrack'

In [6]:
directory_targ

'/projects/0/einf914/data/'

In [4]:
# name_job = f"441Tryone_{time_ns()}.sh"
# name_job = f"chicory"
name_job = f"A5"

run_launcher(
    "dropbox_downloader.py",
    [directory_targ, name_job, 2, "stitcher.py"],
    plates,
    "12:00:00",
    name_job=name_job,
    dependency=True,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 4465664


In [9]:
name_job = f"one_shot_{time_ns()}.sh"
name_job = "fix"

run_launcher(
    "dropbox_downloader.py",
    [directory_targ, name_job, 20, "hypha_identifier.py"],
    plates,
    "12:00:00",
    name_job=name_job,
    dependency=True,
)
# run_launcher('dropbox_uploader.py',[directory_targ,name_job],plates,'40:00',dependency=True,name_job = name_job)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 4144744


In [4]:
name_job = f"one_shot_{time_ns()}.sh"
name_job = "Vasilis"

run_launcher(
    "dropbox_downloader.py",
    [directory_targ, name_job, 20, "skelet_realigner.py"],
    plates,
    "12:00:00",
    name_job=name_job,
    dependency=True,
)
# run_launcher('dropbox_uploader.py',[directory_targ,name_job],plates,'40:00',dependency=True,name_job = name_job)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3908513


In [11]:
name_job = "P3"
run_launcher(
    "hypha_identifier.py",
    [directory_targ, name_job, -1],
    plates,
    "24:00:00",
    dependency=True,
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 4441592


In [13]:
name_job = "transport"
run_launcher(
    "realigner.py",
    [directory_targ, name_job, 20],
    plates,
    "24:00:00",
    dependency=True,
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 4412955


In [7]:
plates

['478_20230814',
 '468_20230809',
 '447_20230821',
 '487_20230922',
 '492_20230901',
 '471_20230821',
 '486_20231009',
 '494_20230908',
 '482_20230908',
 '495_20231013',
 '463_20231013',
 '491_20231013',
 '481_20231005',
 '483_20231005',
 '490_20231003',
 '485_20230929',
 '310_20230830']

In [4]:
name_job = "upload"
run_launcher(
    "analysis_uploader_no_upload.py",
    [directory_targ, name_job, 1],
    plates,
    "6:00:00",
    dependency=True,
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 4346621


In [8]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
len(folders)

258

In [6]:
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]

In [7]:
analysis_folders

,folder_analysis,version,total_path,Plate,strain,medium,CrossDate,date,unique_id,datetime,...,path_time_hypha_info,path_time_plate_info,path_global_plate_info,date_run_analysis,PrincePos,root,split,Pbait,Temperature,Temp
89,Analysis_28_20230227_200_Version1,Version1,/projects/0/einf914/data/20230227_1704_Plate06,28,'Clarum','001P100N100C',20230227,"27.02.2023, 17:04:",28_20230227,2023-02-27 17:04:00,...,Analysis_28_20230227_200_Version1/time_hypha_info,Analysis_28_20230227_200_Version1/time_plate_i...,Analysis_28_20230227_200_Version1/global_plate...,"01.01.1970, 01:00:",6,'Carrot','Y','-',25,nan
140,Analysis_219_20230307_200_Version1,Version1,/projects/0/einf914/data/20230307_1522_Plate11,219,'C3','001P100N100C',20230307,"07.03.2023, 15:22:",219_20230307,2023-03-07 15:22:00,...,Analysis_219_20230307_200_Version1/time_hypha_...,Analysis_219_20230307_200_Version1/time_plate_...,Analysis_219_20230307_200_Version1/global_plat...,"01.01.1970, 01:00:",11,'Carrot','Y','-',25,nan


In [7]:
name_job = "agg_clarum"
folders = folders.loc[folders["/Img/TileConfiguration.txt.registered"] == True]
num_parallel = 100
time = "6:00:00"
hyph_width = 30
perc_low = 85
perc_high = 99.5
minlow = 10
minhigh = 70

args = [hyph_width, perc_low, perc_high, minlow, minhigh, directory_targ]
run_parallel(
    "extract_skel_2.py",
    args,
    folders,
    num_parallel,
    time,
    "skeletonization",
    cpus=128,
    node="fat",
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1.0 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2880366


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1.0 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2880367


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1.0 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


...

Submitted batch job 2880372


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1.0 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2880373
Submitted batch job 2880374


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1.0 node, based on the number of CPUs, GPUs and the amount memory that you've requested.
sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1.0 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2880375


In [5]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
print(len(folders))
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]
print(len(folders))

344
300


In [38]:
for plate in plates:
    folders = all_folders.loc[all_folders["unique_id"] == plate]
    folders1 = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]
    folders2 = folders.loc[folders["/Analysis/skeleton_compressed.mat"] == True]
    print(
        plate,
        len(folders1),
        len(folders2),
        folders1["folder"].sort_values().iloc[0],
        folders2["folder"].sort_values().iloc[0],
    )
    # if plate == '13_20220422':
    #     break

3_20220426 187 286 20220426_1859_Plate16 20220426_1357_Plate16
12_20220502 178 256 20220502_2329_Plate13 20220502_2128_Plate13
13_20220422 93 269 20220507_1235_Plate18 20220422_1514_Plate18
16_20220419 113 268 20220419_2029_Plate10 20220419_1816_Plate10
21_20220502 167 209 20220502_1912_Plate04 20220502_1712_Plate04
480_20221205 342 395 20221205_1912_Plate15 20221205_1712_Plate15
28_20230227 126 169 20230227_1704_Plate06 20230227_1504_Plate06
206_20230303 198 199 20230303_1923_Plate10 20230303_1723_Plate10
202_20230314 237 261 20230314_1801_Plate07 20230314_1608_Plate07
218_20230227 155 184 20230227_1707_Plate07 20230227_1507_Plate07
219_20230307 91 120 20230307_1522_Plate11 20230307_1322_Plate11
229_20230330 235 236 20230330_1553_Plate12 20230330_1352_Plate12


In [36]:
folders1["folder"].sort_values(), folders2["folder"]

(965     20220507_1235_Plate18
 1245    20220507_1435_Plate18
 292     20220507_1635_Plate18
 19      20220507_1835_Plate18
 1792    20220507_2035_Plate18
                 ...          
 1623    20220611_0035_Plate34
 1530    20220611_1235_Plate34
 879     20220612_0035_Plate34
 416     20220612_1235_Plate34
 375     20220613_0035_Plate34
 Name: folder, Length: 93, dtype: object,
 5       20220607_2252_Plate34
 7       20220505_0143_Plate18
 8       20220430_1331_Plate18
 16      20220427_2107_Plate18
 19      20220507_1835_Plate18
                 ...          
 3402    20220518_0225_Plate34
 3423    20220508_1235_Plate18
 3433    20220526_0909_Plate34
 3471    20220430_1531_Plate18
 3519    20220504_1941_Plate18
 Name: folder, Length: 269, dtype: object)

In [4]:
folders = all_folders

In [18]:
name_job = "empty"
dir_drop = "DATA/PRINCE"
delete = True
run_parallel_transfer(
    "toward_drop.py",
    [dir_drop, delete],
    folders,
    70,
    "24:00:00",
    "staging",
    cpus=1,
    node="staging",
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 4444404


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 4444405


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1 CPUs, based on the number of CPUs and the amount memory that you've requested.


...

Submitted batch job 4444443


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 4444444


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 4444445


In [6]:
plates = [plate for plate in all_folders["unique_id"].unique() if plate not in plates]

In [6]:
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    run_launcher,
    run_parallel_all_time,
)

args = [directory_targ]
num_parallel = 32
name_job = "agg_rerun"
time = "6:00:00"
run_parallel_all_time(
    "make_labeled_graphs.py",
    args,
    folders,
    num_parallel,
    time,
    "make_graphs",
    cpus=32,
    node="thin",
    dependency=True,
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2828290


In [10]:
all_folders["unique_id"].unique()

array(['69_20201119', '753_20221013', '764_20221219', '765_20221213',
       '88_20210915', '790_20210907', '757_20221019', '731_20221013',
       '809_20211012', '737_20221025', '799_20211001', '762_20221213',
       '763_20230109', '408_20221013', '734_20221019', '803_20211012',
       '752_20221025', '907_20220321', '403_20221005', '40_20201214',
       '943_20220322'], dtype=object)

In [7]:
folders.loc[folders["/Analysis/transform.mat"] == True]

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp,unique_id,datetime
18,/projects/0/einf914/data/20230106_1749_Plate02,521,2,'Carrot','C2','001P100N100C','Y','-',20230104,25,...,False,True,True,True,True,"06.01.2023, 17:49:",20230106_1749_Plate02,NaN,521_20230104,2023-01-06 17:49:00
25,/projects/0/einf914/data/20230125_0554_Plate02,521,2,'Carrot','C2','001P100N100C','Y','-',20230104,25,...,False,False,False,False,False,"25.01.2023, 05:54:",20230125_0554_Plate02,NaN,521_20230104,2023-01-25 05:54:00
38,/projects/0/einf914/data/20230110_1546_Plate02,521,2,'Carrot','C2','001P100N100C','Y','-',20230104,25,...,False,True,True,True,True,"10.01.2023, 15:46:",20230110_1546_Plate02,NaN,521_20230104,2023-01-10 15:46:00
78,/projects/0/einf914/data/20230123_1554_Plate02,521,2,'Carrot','C2','001P100N100C','Y','-',20230104,25,...,False,False,False,False,False,"23.01.2023, 15:54:",20230123_1554_Plate02,NaN,521_20230104,2023-01-23 15:54:00
101,/projects/0/einf914/data/20230124_2154_Plate02,521,2,'Carrot','C2','001P100N100C','Y','-',20230104,25,...,False,False,False,False,False,"24.01.2023, 21:54:",20230124_2154_Plate02,NaN,521_20230104,2023-01-24 21:54:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679,/projects/0/einf914/data/20230112_0549_Plate02,521,2,'Carrot','C2','001P100N100C','Y','-',20230104,25,...,False,True,True,True,True,"12.01.2023, 05:49:",20230112_0549_Plate02,NaN,521_20230104,2023-01-12 05:49:00
1685,/projects/0/einf914/data/20230109_1409_Plate02,521,2,'Carrot','C2','001P100N100C','Y','-',20230104,25,...,False,True,True,True,True,"09.01.2023, 14:09:",20230109_1409_Plate02,NaN,521_20230104,2023-01-09 14:09:00
1700,/projects/0/einf914/data/20230105_0833_Plate02,521,2,'Carrot','C2','001P100N100C','Y','-',20230104,25,...,False,True,True,True,True,"05.01.2023, 08:33:",20230105_0833_Plate02,NaN,521_20230104,2023-01-05 08:33:00
1737,/projects/0/einf914/data/20230124_0954_Plate02,521,2,'Carrot','C2','001P100N100C','Y','-',20230104,25,...,False,False,False,False,False,"24.01.2023, 09:54:",20230124_0954_Plate02,NaN,521_20230104,2023-01-24 09:54:00


In [12]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

name_job = "malin"
stage = 30
args = [directory_targ]
for plate in plates:
    select = folders.loc[folders["unique_id"] == plate]
    num_parallel = 32
    time = "10:00"
    args = [directory_targ]
    run_parallel(
        "realign.py",
        args,
        select,
        num_parallel,
        time,
        "create_realign",
        cpus=32,
        node="fat",
        name_job=name_job,
    )

run_launcher(
    "graph_extracter.py",
    [directory_targ, name_job, stage - 1],
    plates,
    "3:00:00",
    dependency=True,
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1872208


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1872209


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


...

Submitted batch job 1872314


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1872315


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


Submitted batch job 1872316


In [7]:
name_job = "rerun"
run_launcher(
    "post_processer_1.py",
    [directory_targ, name_job, 10],
    plates,
    "3:00:00",
    dependency=True,
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2143049


In [8]:
from amftrack.util.dbx import upload_folder

update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
dir_drop = "DATA/PRINCE"

for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    id_unique = row["unique_id"]
    path = os.path.join(directory_targ, folder)
    target_drop = f"/{dir_drop}/{id_unique}/{folder}"
    print(dir_drop)
    upload_folder(path, target_drop, delete=False)

/projects/0/einf914/data/Analysis_93_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_5_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_69_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_26_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_32_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_43_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_101_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_42_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_15_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_65_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_28_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_39_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_108_20220921_200_Version1/folder_info.json
/projects/0

In [16]:
from amftrack.pipeline.functions.post_processing.exp_plot import *
from amftrack.pipeline.launching.run_super import run_parallel_post

name_job = "malin"
time = "1:00:00"
list_f = [plot_tracking]
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
directory = directory_targ
list_args = [[]] * len(list_f)
overwrite = True
num_parallel = 100
run_parallel_post(
    "exp_plot.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "global_plate_post_process",
    cpus=128,
    name_job=name_job,
    node="fat",
)

/projects/0/einf914/data/Analysis_101_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_42_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_39_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_108_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_67_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_56_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_50_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_23_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_21_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_100_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_99_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_82_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_61_20220921_200_Version1/folder_info.json
/projects

In [15]:
from amftrack.pipeline.functions.post_processing.time_hypha import *
import pandas as pd
import os

update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
list_f = [
    get_pos_x,
    get_pos_y,
    get_time_since_begin_exp,
    get_distance_final_pos,
    get_timedelta,
    get_time_since_start,
    get_speed,
    get_timestep,
    get_timestep_init,
    get_time_init,
    get_degree,
    get_width_tip_edge,
    get_width_root_edge,
    get_width_average,
    get_has_reached_final_pos,
    get_in_ROI,
]
# list_f = [local_density,local_density,local_density]
# list_f = [get_time_since_begin_exp]
# list_f = [get_width_tip_edge, get_width_root_edge]
list_args = [{}] * len(list_f)
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = True
load_graphs = True
num_parallel = 32
time = "10:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "time_hypha_post_process.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "time_hypha_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
        )

/projects/0/einf914/data/Analysis_101_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_42_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_39_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_108_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_67_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_56_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_50_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_23_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_21_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_100_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_99_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_82_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_61_20220921_200_Version1/folder_info.json
/projects

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1867673
Submitted batch job 1867674


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1867676


...

Submitted batch job 1867715


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1867716


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1867717


In [36]:
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    run_launcher,
    run_parallel_all_time,
)

folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]
args = [directory_targ]
num_parallel = 32
time = "1:00:00"
limit = 200
version = 1
suffix = "_labeled"
lim_considered = 10
args = [directory_targ, limit, version, suffix, lim_considered]
run_parallel_all_time(
    "hyphae_extraction.py",
    args,
    folders,
    num_parallel,
    time,
    "hyphae",
    cpus=32,
    node="fat",
    name_job="test",
)

Submitted batch job 1857991


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1857992


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1857995


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1857997


In [14]:
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]
len(folders)

451

In [20]:
len(list(set(folders["unique_id"].values)))

107

In [12]:
stage = 20
run_launcher(
    "post_processer_1.py",
    [directory_targ, name_job, stage - 1],
    plates,
    "20:00",
    dependency=True,
    name_job="test",
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2061667


In [28]:
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]

/projects/0/einf914/data/Analysis_93_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_92_20220921_2_Version1/folder_info.json
/projects/0/einf914/data/Analysis_95_20220921_2_Version1/folder_info.json
/projects/0/einf914/data/Analysis_94_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_92_20220921_4_Version1/folder_info.json
/projects/0/einf914/data/Analysis_99_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_95_20220921_3_Version1/folder_info.json
/projects/0/einf914/data/Analysis_94_20220921_3_Version1/folder_info.json
/projects/0/einf914/data/Analysis_94_20220921_1_Version1/folder_info.json
/projects/0/einf914/data/Analysis_93_20220921_3_Version1/folder_info.json
/projects/0/einf914/data/Analysis_90_20220921_2_Version1/folder_info.json
/projects/0/einf914/data/Analysis_98_20220921_3_Version1/folder_info.json
/projects/0/einf914/data/Analysis_91_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analy

In [7]:
analysis_folders

,folder_analysis,version,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,...,datetime,date_begin,date_end,number_timepoints,path_exp,path_global_hypha_info,path_time_hypha_info,path_time_plate_info,path_global_plate_info,date_run_analysis
0,Analysis_33_20220502_188_Version1,Version1,/projects/0/einf914/data/20220502_2111_Plate07,33,7,'Carrot','Agg','001P100N100C','Y','-',...,2022-05-02 21:11:00,"02.05.2022, 21:11:","20.05.2022, 09:41:",188,Analysis_33_20220502_188_Version1/experiment.pick,Analysis_33_20220502_188_Version1/global_hypha...,Analysis_33_20220502_188_Version1/time_hypha_info,Analysis_33_20220502_188_Version1/time_plate_i...,Analysis_33_20220502_188_Version1/global_plate...,"01.01.1970, 01:00:"


In [8]:
# name_job = f"one_shot_{time_ns()}.sh"

from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.global_plate import *
from amftrack.pipeline.functions.post_processing.time_plate import *
from amftrack.pipeline.functions.post_processing.global_hypha import *
from amftrack.pipeline.functions.post_processing.area_hulls import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
from amftrack.pipeline.functions.post_processing.exp_plot import *

time = "24:00:00"
directory = directory_targ
max_ind = 20
incr = 100
fs = [
    get_density_in_ring,
    get_density_anastomose_in_ring,
    # get_density_branch_rate_in_ring,
    # get_density_stop_rate_in_ring,
    get_density_active_tips_in_ring,
]
# fs = [get_mean_speed_in_ring]

list_f = []
list_args = []

for f in fs:
    list_f += [f] * max_ind

    list_args += [{"incr": incr, "i": i, "rh_only": True} for i in range(max_ind)]
overwrite = False
num_parallel = 30
run_parallel_post(
    "time_plate_post_process.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "time_plate_post_process",
    cpus=32,
    name_job=name_job,
    node="fat",
    dependency=True,
)
run_launcher(
    "analysis_uploader.py",
    [directory_targ, name_job],
    plates,
    "12:00:00",
    dependency=True,
    name_job=name_job,
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1768604


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


Submitted batch job 1768605


In [41]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

name_job = "malin"

num_parallel = 100
time = "20:00"
args = [directory_targ]
run_parallel(
    "compress_image.py",
    args,
    folders,
    num_parallel,
    time,
    "compress",
    cpus=128,
    node="fat",
    name_job=name_job,
)

Submitted batch job 1858250
Submitted batch job 1858251
Submitted batch job 1858252
Submitted batch job 1858253
Submitted batch job 1858255
Submitted batch job 1858256


In [42]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

num_parallel = 50
time = "12:00:00"
hyph_width = 30
perc_low = 93
perc_high = 99.5
args = [hyph_width, perc_low, perc_high, directory_targ]
run_parallel(
    "extract_skel_2.py",
    args,
    folders,
    num_parallel,
    time,
    "skeletonization",
    cpus=128,
    node="fat",
    name_job=name_job,
)

Submitted batch job 1858259
Submitted batch job 1858260
Submitted batch job 1858262
Submitted batch job 1858267
Submitted batch job 1858268
Submitted batch job 1858269
Submitted batch job 1858270
Submitted batch job 1858271
Submitted batch job 1858272
Submitted batch job 1858273
Submitted batch job 1858274
Submitted batch job 1858275


In [43]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

num_parallel = 100
time = "20:00"
args = [directory_targ]
run_parallel(
    "detect_blob.py",
    args,
    folders,
    num_parallel,
    time,
    "detect_blob",
    cpus=128,
    node="fat",
    name_job=name_job,
)

Submitted batch job 1858277
Submitted batch job 1858279
Submitted batch job 1858282
Submitted batch job 1858283
Submitted batch job 1858284
Submitted batch job 1858285


In [7]:
name_job = "malin"

run_launcher(
    "masker.py",
    [directory_targ, name_job, 30],
    plates,
    "3:00:00",
    dependency=True,
    name_job=name_job,
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


Submitted batch job 1861160


In [25]:
from amftrack.pipeline.launching.run_super import run_parallel_all_time, run_launcher

folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Img/TileConfiguration.txt.registered"] == True]
num_parallel = 50
time = "10:00"
args = []
run_parallel_all_time(
    "make_video_stitched.py",
    args,
    folders,
    num_parallel,
    time,
    "make_video",
    cpus=32,
    node="fat",
    dependency=False,
    name_job="video",
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1828129


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1828130


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1828131


In [3]:
plates = [
    "105_20220817",
    "77_20220808",
    "71_20220817",
    "100_20220812",
    "117_20220815",
    "99_20220815",
    "75_20220808",
    "226_20220812",
]

In [46]:
folders = all_folders.loc[all_folders["Plate"].isin(["32"])]

In [17]:
from amftrack.pipeline.launching.run_super import *

num_parallel = 32
time = "8:00:00"
limit = 200
version = 1
labeled = True
args = [directory_targ, limit, version, labeled]
run_parallel_all_time(
    "hyphae_extraction.py",
    args,
    folders,
    num_parallel,
    time,
    "hyphae",
    cpus=32,
    node="fat",
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1738329


In [12]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

for plate in plates:
    select = folders.loc[folders["Plate"] == plate]
    num_parallel = 128
    time = "12:00:00"
    thresh = 10000  # For R. irregularis, thresh 10000 is good. For Aggregatum, higher may be necessary
    args = [thresh, directory_targ]
    run_parallel(
        "final_alignment.py",
        args,
        select,
        num_parallel,
        time,
        "realign",
        cpus=128,
        node="fat",
        name_job=name_job,
    )

Submitted batch job 1138954


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [20]:
plates

['6_20220509']

In [19]:
select

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime


In [13]:
i = 0
run_info = folders
folder_list = list(run_info["folder"])
folder_list.sort()

dates_datetime_chosen = folder_list[i : i + 2]
print("========")
print(f"Matching plate {dates_datetime_chosen[0]} at dates {dates_datetime_chosen}")
print("========")
dates = dates_datetime_chosen

Matching plate 20220510_1851_Plate17 at dates ['20220510_1851_Plate17', '20220511_1750_Plate17']


In [10]:
from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.global_plate import *
from amftrack.pipeline.functions.post_processing.time_plate import *
from amftrack.pipeline.functions.post_processing.global_hypha import *
from amftrack.pipeline.functions.post_processing.area_hulls import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
from amftrack.pipeline.functions.post_processing.exp_plot import *

update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
directory = directory_targ

In [11]:
time = "3:40:00"
directory = directory_targ
# list_f = [get_width_f,get_tot_length_C_f,get_tot_growth_C_f]
# list_f = [get_timestep_anastomosis]
list_f = [
    gets_out_of_ROI,
    get_width_f,
    get_tot_length_C_f,
    get_tot_growth_C_f,
    get_tot_length_pp_f,
    get_tot_growth_pp_f,
    get_timestep_stop_growth,
    get_time_stop_growth,
    get_time_init_growth,
    get_mean_speed_growth,
    get_stop_track,
    get_timestep_anastomosis,
    get_timestep_biological_stop_growth,
]
# list_f = [get_num_branch]
# list_f = [gets_out_of_ROI]
list_args = [{}] * len(list_f)
overwrite = False
num_parallel = 32
run_parallel_post(
    "global_hypha_post_process.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "global_hypha_post_process",
    cpus=32,
    name_job="a",
    node="fat",
)

Submitted batch job 1539552


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [13]:
import sys
from amftrack.util.sys import (
    update_analysis_info,
    get_analysis_info,
)
from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
import pandas as pd
import os

list_f = [
    get_pos_x,
    get_pos_y,
    get_time_since_begin_exp,
    get_distance_final_pos,
    get_timedelta,
    get_time_since_start,
    get_speed,
    get_timestep,
    get_timestep_init,
    get_time_init,
    get_degree,
    get_width_tip_edge,
    get_width_root_edge,
    get_width_average,
    has_reached_final_pos,
    in_ROI,
]
# list_f = [local_density,local_density,local_density]
# list_f = [get_time_since_begin_exp]
# list_f = [get_width_tip_edge, get_width_root_edge]
list_args = [{}] * len(list_f)
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = False
load_graphs = True
num_parallel = 32
time = "12:00:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "time_hypha_post_process.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "time_hypha_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
        )

Submitted batch job 1539625
Submitted batch job 1539626
Submitted batch job 1539627


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memor

Submitted batch job 1539628
Submitted batch job 1539629
Submitted batch job 1539630


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1539631


...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1539645
Submitted batch job 1539646


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1539647
Submitted batch job 1539648
Submitted batch job 1539649


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
